# Getting Spotify API Token

In [47]:
import requests
import base64
import datetime

In [48]:
client_id = 'c034711d4add45bbb45b39ab2f8d0d7b'
client_secret = '0581e69388974a3ebe0335c4d2ee4593'

# Spotify API Class

In [72]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    expired = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
        
    def get_client_credentials(self):
        '''
        Return a base-64 encoded string
        '''
        
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret.")
        
        
        client_creds = f"{self.client_id}:{self.client_secret}"
            
        client_creds_base64 = base64.b64encode(client_creds.encode())
        
        return client_creds_base64.decode()
        
    def get_token_headers(self):
        
        client_creds_base64 = self.get_client_credentials()
        
        token_headers = {
    "Authorization":f"Basic {client_creds_base64}", #base 64 encoded string
}
        
        return token_headers
    
    def get_token_data(self):
        
        token_data = {
    "grant_type":"client_credentials"
}
        
        return token_data
    
    def perform_auth(self):
        
        r = requests.post(self.token_url, data = self.get_token_data(), headers = self.get_token_headers())
        token_response_data = r.json()
        
        if r.status_code not in range(200, 299):
            return False
        
        now = datetime.datetime.now()
        self.access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token_expires = expires
        self.expired = expires < now
        return True
    
    def get_token(self):
        return self.access_token

In [73]:
spotify = SpotifyAPI(client_id, client_secret)

In [76]:
spotify.perform_auth()

True

In [78]:
access_token = spotify.get_token()

In [80]:
access_token

'BQDsBZHmtuJbvnjY-EkLj6ib94I6yn9aMebHOYkR0IlJq-rVif8rdGsKqZB_VbWrYYRi9BJb6Zgx3gkIrVg'

In [83]:
headers = {
    "Authorization": f"Bearer{access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = {"q": "Time", "type": "track"}
r = requests.get(endpoint, data = data, headers = headers)

In [84]:
r.text

'<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 400 (Bad Request)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-moz-border-image:u